In [4]:
import pandas as pd

In [5]:
xls = pd.ExcelFile('Prueba.xlsx')

In [6]:
sheet_names = xls.sheet_names
sheet_names

['Instrucciones', 'Glosario', 'PuntoPartida', 'PuntoLlegada', 'Siniestros']

In [7]:
instructions_df = pd.read_excel(xls, 'Instrucciones')
glossary_df = pd.read_excel(xls, 'Glosario')
punto_partida_df = pd.read_excel(xls, 'PuntoPartida')
punto_llegada_df = pd.read_excel(xls, 'PuntoLlegada')
siniestros_df = pd.read_excel(xls, 'Siniestros')

{'Instructions':    Unnamed: 0                                         Unnamed: 1
 0         NaN  Este ejercicio tiene como objetivo medir las c...
 1         NaN  Para resolver el ejercicio es obligatorio util...
 2         NaN                                                NaN
 3         NaN                                               Tips
 4         NaN  a) Revisa las veces que sea necesario los dicc...,
 'Glossary':    Unnamed: 0     Unnamed: 1                                       Unnamed: 2  \
 0         NaN           Hoja                                      Descripción   
 1         NaN  Instrucciones        Contiene los requisitos de este ejercicio   
 2         NaN       Glosario  Contiene el glosario de la información a tratar   
 3         NaN            NaN                                              NaN   
 4         NaN            NaN                                              NaN   
 
     Unnamed: 3                               Unnamed: 4  
 0      Columna       